In [13]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import os
#from paths import *
import requests
import pandas as pd
import xarray as xr
from os import listdir
from datetime import datetime, timedelta, date
import glob

from scipy.spatial import distance
import holoviews as hv


# Initialize the Earth Engine module
ee.Initialize()

In [6]:
!pwd

/nfs/attic/dfh/Aragon2/Notebooks/preprocess_python


In [7]:
geotiff_list = glob.glob('/nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/GEE_Downloads/*.tif')
geotiff_list

['/nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/GEE_Downloads/2020100100.tif',
 '/nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/GEE_Downloads/2021080106.tif',
 '/nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/GEE_Downloads/2021080212.tif',
 '/nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/GEE_Downloads/2021080118.tif',
 '/nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/GEE_Downloads/2021080200.tif',
 '/nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/GEE_Downloads/2021080218.tif',
 '/nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/GEE_Downloads/2021080206.tif',
 '/nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/GEE_Downloads/2011080100.tif',
 '/nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/GEE_Downloads/2021080100.tif',
 '/nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/GEE_Downloads/2021080112.tif']

In [8]:
#########################################################################
############################ USER INPUTS ################################
#########################################################################
# DOMAIN
# choose the modeling domain
domain = 'WY'
print(domain)

domains_resp = requests.get("https://raw.githubusercontent.com/snowmodel-tools/preprocess_python/master/CSO_domains.json")
domains = domains_resp.json()
    
# PATHS
# path to store tif files from gee
TIFpath = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/GEE/'
# # path to where you want your output met .dat fime
# OUTpath = 'OR_met_test.dat'#'/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/mm_'+domain+'_2011-2016.dat'

# TIME
# choose if want to set 'manual' or 'auto' date 
date_flag = 'manual'
# If you choose 'manual' set your dates below  
# This will start on the 'begin' date at 0:00 and the last iteration will 
# be on the day before the 'end' date below.
st_dt = '2011-09-01'#domains[domain]['st']
ed_dt = '2011-09-10'#domains[domain]['ed']
#########################################################################

WY


In [9]:
# Date setup function
def set_dates(st_dt,ed_dt,date_flag):
    if date_flag == 'auto':
        # ###automatically select date based on today's date 
        hoy = date.today()
        antes = timedelta(days = 2)
        #end date 3 days before today's date
        fecha = hoy - antes
        eddt = fecha.strftime("%Y-%m-%d") 
        #start date
        if fecha.month <10:
            styr = fecha.year - 1
        else:
            styr = fecha.year
        stdt = str(styr)+'-10-01'
    elif date_flag == 'manual':
        stdt = st_dt
        eddt = ed_dt
    return stdt, eddt

In [18]:
# Download CFSv2 met data function
def get_cfsv2(domain, TIFpath, stdt, eddt):
    # in GEE the last iteration is on the day before the 'end' date below
    # we adjust this here since it is not intuative
    #eddt = (datetime.strptime(eddt, '%Y-%m-%d')+timedelta(days = 1)).strftime('%Y-%m-%d')
    
    #create directory with initiation date for ensemble if it doesn't exist
    get_ipython().system('mkdir -p $TIFpath')

    #path to CSO domains
    domains_resp = requests.get("https://raw.githubusercontent.com/snowmodel-tools/preprocess_python/master/CSO_domains.json")
    domains = domains_resp.json()

    '''
    // These are the min and max corners of your domain in Lat, Long
    // Western Wyoming
    // Input the minimum lat, lower left corner
    '''
    minLat = domains[domain]['Bbox']['latmin']
    #// Input the minimum long, lower left corner
    minLong = domains[domain]['Bbox']['lonmin']
    #// Input the max lat, upper right corner
    maxLat = domains[domain]['Bbox']['latmax']
    #// Input the max Long, upper right corner
    maxLong = domains[domain]['Bbox']['lonmax']

    #/ These are the min and max corners of your reanalysis in Lat, Long (create a slightly larger box)
    #// Input the minimum lat, lower left corner
    minLatMET = (minLat - 0.25);
    #// print(minLat2);
    #// Input the minimum long, lower left corner
    minLongMET = (minLong - 0.5);
    #// Input the max lat, upper right corner
    maxLatMET = (maxLat + 0.25);
    #// Input the max Long, upper right corner
    maxLongMET = (maxLong + 0.5);

    # This resolution for the NLCD and DEM outputs for the SnowModel domain in meters
    sm_resolution = int(domains[domain]['cellsize'])

    '''// Resolution for the PRISM output. This shoud change by Latitude of the domain
    // because the PRISM product spatial resolution is 2.5 minutes, which equals 150 arc seconds.
    // You can use this arc-second calculator to estimate the correct value for the PRISM resolution by latitude
    // https://opendem.info/arc2meters.html
    // This is one arc-second in meters for 43 degrees N Latitude'''
    one_arcsecond = 22.57
    PRISM_resolution = one_arcsecond * 150

    '''// Define the final output projection using EPSG codes'''
    epsg_code = domains[domain]['mod_proj']

    #// Name the DEM output
    dem_name = 'DEM'
    #// Name the Land Cover output
    lc_name = 'NLCD2016'

    my_domain = ee.Geometry.Rectangle(**{'coords':[minLong,minLat,maxLong,maxLat],'proj': 'EPSG:4326','geodesic':True,});
    my_domain_met = ee.Geometry.Rectangle([minLongMET,minLatMET,maxLongMET,maxLatMET])
    
    # download reanalysis data
    cfsv2 = ee.ImageCollection('NOAA/CFSV2/FOR6H').filterBounds(my_domain_met).filter(ee.Filter.date(stdt,eddt))

    temp = cfsv2.select('Temperature_height_above_ground')
    temp_dir = TIFpath+'GEE_temp/'
    get_ipython().system('mkdir -p $temp_dir')
    geemap.ee_export_image_collection(temp, out_dir=temp_dir,region=my_domain_met,scale=22200,crs=epsg_code)
    
    prcp = cfsv2.select('Precipitation_rate_surface_6_Hour_Average')
    prcp_dir = TIFpath+'GEE_pr/'
    get_ipython().system('mkdir -p $prcp_dir')
    geemap.ee_export_image_collection(prcp, out_dir=prcp_dir,region=my_domain_met,scale=22200,crs=epsg_code)
    
    return temp_dir, prcp_dir

In [19]:
stdt, eddt=set_dates(st_dt,ed_dt,date_flag)
temp_dir, prcp_dir = get_cfsv2(domain, TIFpath, stdt, eddt)

Total number of images: 36

Exporting 1/36: 2011090100.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/Aragon2/CSOdmn/WY/GEE/GEE_temp/2011090100.tif


Exporting 2/36: 2011090106.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/Aragon2/CSOdmn/WY/GEE/GEE_temp/2011090106.tif


Exporting 3/36: 2011090112.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/Aragon2/CSOdmn/WY/GEE/GEE_temp/2011090112.tif


Exporting 4/36: 2011090118.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/Aragon2/CSOdmn/WY/GEE/GEE_temp/2011090118.tif


Exporting 5/36: 2011090200.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/Aragon2/CSOdmn/WY/GEE/GEE_temp/2011090200.tif


Exporting 6/36: 2011090206.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/Aragon2/CSOdmn/WY/GEE/GEE_temp/2011090206.tif


Exporting 7/36: 2011090212.tif
Generating URL ...
Please wait ...
Data downloaded to

In [33]:
geotiff_list = glob.glob(temp_dir+'*.tif')
#https://docs.dea.ga.gov.au/notebooks/Frequently_used_code/Opening_GeoTIFFs_NetCDFs.html
# Create variable used for time axis
time_var = xr.Variable('time',[pd.to_datetime(i[-14:-4], format='%Y%m%d%H') for i in geotiff_list])

# Load in and concatenate all individual GeoTIFFs
geotiffs_da = xr.concat([xr.open_rasterio(i) for i in geotiff_list],
                        dim=time_var)
#convert units to C
geotiffs_da = geotiffs_da-273.16

# Covert our xarray.DataArray into a xarray.Dataset
met_temp = geotiffs_da.to_dataset('band')

# Rename the variable to a more useful name
met_temp = met_temp.rename({1: 'Temperature_C'})

# Print the output
met_temp

<xarray.Dataset>
Dimensions:        (time: 36, x: 11, y: 14)
Coordinates:
  * y              (y) float64 4.962e+06 4.94e+06 ... 4.695e+06 4.673e+06
  * x              (x) float64 4.551e+05 4.773e+05 ... 6.549e+05 6.771e+05
  * time           (time) datetime64[ns] 2011-09-03T06:00:00 ... 2011-09-07
Data variables:
    Temperature_C  (time, y, x) float32 1.259 1.307 0.609 ... 11.06 10.13 9.286

In [35]:
geotiff_list = glob.glob(prcp_dir+'*.tif')

# Create variable used for time axis
time_var = xr.Variable('time',[pd.to_datetime(i[-14:-4], format='%Y%m%d%H') for i in geotiff_list])

# Load in and concatenate all individual GeoTIFFs
geotiffs_da = xr.concat([xr.open_rasterio(i) for i in geotiff_list],
                        dim=time_var)

#convert precip rate to precip DEPTH (mm) during time interval
geotiffs_da=geotiffs_da*24*3600/4

# Covert our xarray.DataArray into a xarray.Dataset
met_pr = geotiffs_da.to_dataset('band')

# Rename the variable to a more useful name
met_pr = met_pr.rename({1: 'Precipitation_mm'})

# Print the output
met_pr

<xarray.Dataset>
Dimensions:           (time: 36, x: 11, y: 14)
Coordinates:
  * y                 (y) float64 4.962e+06 4.94e+06 ... 4.695e+06 4.673e+06
  * x                 (x) float64 4.551e+05 4.773e+05 ... 6.549e+05 6.771e+05
  * time              (time) datetime64[ns] 2011-09-03T06:00:00 ... 2011-09-07
Data variables:
    Precipitation_mm  (time, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0216

# now try to use holoviews to compare the data 

In [ ]:
def nearest_grid(ds, pt):
    
    """
    Returns the nearest lon and lat to pt in a given Dataset (ds).
    
    pt : input point, tuple (longitude, latitude)
    output:
        lon, lat
    """
    
    if all(coord in list(ds.coords) for coord in ['lat', 'lon']):
        df_loc = ds[['lon', 'lat']].to_dataframe().reset_index()
    else:
        df_loc = ds[['orig_lon', 'orig_lat']].isel(time=0).to_dataframe().reset_index()
    
    loc_valid = df_loc.dropna()
    pts = loc_valid[['lon', 'lat']].to_numpy()
    idx = distance.cdist([pt], pts).argmin()
    
    return loc_valid['lon'].iloc[idx], loc_valid['lat'].iloc[idx]

In [ ]:
# get lon, lat of snodas grid cell nearest to the LIS coordinates we used earlier
snodas_ts_lon, snodas_ts_lat = nearest_grid(snodas_depth_ds, (ts_lon, ts_lat))

# define a date range to plot (shorter = quicker for demo)
start_date, end_date = ('2018-01-01', '2018-03-01')
plot_daterange = slice(start_date, end_date)

# select SNODAS grid cell and subset to plot_daterange
snodas_snd_subset_ds = snodas_depth_ds.sel(lon=snodas_ts_lon,
                                             lat=snodas_ts_lat,
                                             time=plot_daterange)

# select LIS grid cell and subset to plot_daterange
lis_snd_subset_ds = lis_output_ds['SnowDepth_tavg'].sel(lat=ts_lat,
                                                        lon=ts_lon,
                                                        time=plot_daterange)

# create SNODAS snow depth plot
snodas_snd_plot = snodas_snd_subset_ds.hvplot(label='SNODAS')

# create LIS snow depth plot
lis_snd_plot = lis_snd_subset_ds.hvplot(label='LIS')

# create SNODAS vs LIS snow depth plot
lis_vs_snodas_snd_plot = (lis_snd_plot * snodas_snd_plot)

# display the plot
lis_vs_snodas_snd_plot.opts(title=f'Snow Depth @ Lon: {ts_lon}, Lat: {ts_lat}',
                            legend_position='right',
                            xlabel='Date',
                            ylabel='Snow Depth (m)')

In [36]:
from scipy.spatial import distance